In [39]:
"""
Clean the transfermarket data

Operations:
    - Dedupe arrivals / departures by player
    - Remove any transfers with 
"""
import pandas as pd
import numpy as np

ARRIVALS_FILE = '../scraper/output/arrivals.p'
DEPARTURES_FILE = '../scraper/output/departures.p'

arrivals = pd.read_pickle(ARRIVALS_FILE)
departures = pd.read_pickle(DEPARTURES_FILE)

# Merge arrivals and departures
def merge_arrivals_departures(arrivals, departures):
    # Join on player name and year
    all = arrivals.merge(departures, left_on=['Arrivals','year'], 
                    right_on=['Departures', 'year'], how='outer', suffixes=['_a', '_d'])

    # Some players transfer multiple times per year so we need to also check for 
    # consistency in the team names
    both_transfers = all[(all['index_a'].notnull() & all['index_d'].notnull())]
    from_match = (both_transfers['Moving from'] == both_transfers['team_name_d'])
    to_match = (both_transfers['Moving to'] == both_transfers['team_name_a'])
    drop_rows = both_transfers[~(from_match & to_match)].index
    all = all.drop(drop_rows).reset_index(drop=True)
    return all

all = merge_arrivals_departures(arrivals, departures)

def check_value_consistency(df, column_1, column_2):
    """Ensure that if both column 1 and 2 exist that their values match"""
    has_both = (df[column_1].notnull() & df[column_2].notnull())
    both = df[has_both]
    try:
        assert (both[column_1] == both[column_2]).all() == True
    except:
        return both[both[column_1] != both[column_2]].head()



# integrity checks
check_value_consistency(all, 'Arrivals', 'Departures')
check_value_consistency(all, 'Market value_a', 'Market value_d')
check_value_consistency(all, 'Transfer fee_a', 'Transfer fee_d')
check_value_consistency(all, 'Pos_a', 'Pos_d')
check_value_consistency(all, 'Pos_a', 'Pos_d')

# TODO: Merge duplicate columns
def merge_columns(df, col_1, col_2, new_col):
    """Take first value that exists from col1, col2 and add it to the new column
    Deletes the old columns
    """
    df[new_col] = np.where(df[col_1].isnull(), df[col_2], df[col_1])
    df = df.drop([col_1, col_2], axis=1)
    return df

all = merge_columns(all, 'Arrivals', 'Departures', 'Name')
all = merge_columns(all, 'Market value_a', 'Market value_d', 'Market value')
all = merge_columns(all, 'Transfer fee_a', 'Transfer fee_d', 'Transfer fee')
all = merge_columns(all, 'Pos_a', 'Pos_d', 'Position')




# Drop all observations where there's no market value

# TODO: output pickle file
all.to_pickle("../data/clean_transfermarket.p")


In [45]:
all['Market value'].value_counts()

1,00 Mill. €      292
2,00 Mill. €      242
1,50 Mill. €      225
3,00 Mill. €      210
-                 200
800 Th. €         159
2,50 Mill. €      157
500 Th. €         153
4,00 Mill. €      135
5,00 Mill. €      115
300 Th. €         111
600 Th. €          90
400 Th. €          90
6,00 Mill. €       84
200 Th. €          73
1,20 Mill. €       72
700 Th. €          70
1,80 Mill. €       65
10,00 Mill. €      62
8,00 Mill. €       58
3,50 Mill. €       58
100 Th. €          57
7,00 Mill. €       54
900 Th. €          49
12,00 Mill. €      40
1,60 Mill. €       39
9,00 Mill. €       32
1,40 Mill. €       29
7,50 Mill. €       26
15,00 Mill. €      26
                 ... 
19,00 Mill. €       1
46,00 Mill. €       1
150 Th. €           1
80 Th. €            1
29,00 Mill. €       1
6,40 Mill. €        1
34,00 Mill. €       1
2,90 Mill. €        1
42,00 Mill. €       1
5,60 Mill. €        1
5,25 Mill. €        1
4,90 Mill. €        1
75 Th. €            1
5,70 Mill. €        1
6,70 Mill.

In [43]:
all['Transfer fee'].value_counts()

Free transfer     902
Loan              666
End of loan       472
-                 414
Loan fee:         136
?                  97
1,00 Mill. €       36
3,00 Mill. €       34
1,50 Mill. €       31
5,00 Mill. €       28
2,50 Mill. €       26
4,00 Mill. €       26
6,00 Mill. €       26
2,00 Mill. €       23
10,00 Mill. €      21
3,50 Mill. €       20
8,00 Mill. €       20
500 Th. €          17
7,00 Mill. €       17
300 Th. €          16
1,20 Mill. €       13
15,00 Mill. €      12
30,00 Mill. €      11
600 Th. €          11
4,50 Mill. €       10
25,00 Mill. €      10
20,00 Mill. €      10
5,50 Mill. €        9
400 Th. €           8
750 Th. €           8
                 ... 
2,05 Mill. €        1
17,26 Mill. €       1
3,25 Mill. €        1
4,88 Mill. €        1
35,50 Mill. €       1
65,00 Mill. €       1
2,10 Mill. €        1
8,85 Mill. €        1
2,87 Mill. €        1
5,70 Mill. €        1
35,40 Mill. €       1
88,20 Mill. €       1
4,80 Mill. €        1
230 Th. €           1
320 Th. € 